In [1]:
import pandas as pd

path_items = '../data/processed/df_items_clean.parquet'
path_genres = '../data/processed/df_genres_dummies.parquet'
path_games = '../data/processed/df_games_clean.parquet'

# Cargar los DataFrames desde los archivos Parquet
df_items = pd.read_parquet(path_items)
df_genres = pd.read_parquet(path_genres)
df_games = pd.read_parquet(path_games)

In [2]:
# Ensure 'game_id' is the same data type across all dataframes
df_games['game_id'] = df_games['game_id'].astype('Int64')
df_items['game_id'] = df_items['game_id'].astype('Int64')
df_genres['game_id'] = df_genres['game_id'].astype('Int64')


**Function 1:**
- `def` PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [3]:
# Merge the dataframes on 'game_id'
df_merged = pd.merge(df_items, df_games, on='game_id')
df_merged = pd.merge(df_merged, df_genres, on='game_id')

In [4]:
df_merged.head()

,user_id,items_count,game_id,item_name,playtime_forever,developer,app_name,release_year,Accounting,Action,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,76561197970982479,277,10,Counter-Strike,6.0,Valve,Counter-Strike,2000,0,1,...,0,0,0,0,0,0,1,0,0,0
1,js41637,888,10,Counter-Strike,0.0,Valve,Counter-Strike,2000,0,1,...,0,0,0,0,0,0,1,0,0,0
2,Riot-Punch,328,10,Counter-Strike,0.0,Valve,Counter-Strike,2000,0,1,...,0,0,0,0,0,0,1,0,0,0
3,doctr,541,10,Counter-Strike,93.0,Valve,Counter-Strike,2000,0,1,...,0,0,0,0,0,0,1,0,0,0
4,corrupted_soul,115,10,Counter-Strike,108.0,Valve,Counter-Strike,2000,0,1,...,0,0,0,0,0,0,1,0,0,0


In [5]:
# Function to find the year with the most playtime for a given genre
def PlayTimeGenre(genre, df):
    genre_df = df[df[genre] == 1]
    playtime_by_year = genre_df.groupby('release_year')['playtime_forever'].sum().reset_index()
    max_playtime_year = playtime_by_year.loc[playtime_by_year['playtime_forever'].idxmax()]['release_year']
    max_playtime_year = int(max_playtime_year)  
    return {"Año de lanzamiento con más horas jugadas para Género": genre, "Año": max_playtime_year}


In [6]:
# Example usage of PlayTimeGenre function
print(PlayTimeGenre('Education', df_merged))


{'Año de lanzamiento con más horas jugadas para Género': 'Education', 'Año': 2010}


In [7]:
# Function to list all genres in the dataframe
def list_genres(df):
    return df.columns.tolist()[1:]

# Example usage of list_genres function
print("List of genres:", list_genres(df_genres))

List of genres: ['Accounting', 'Action', 'Adventure', 'Animation & Modeling', 'Audio Production', 'Casual', 'Design & Illustration', 'Early Access', 'Education', 'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports', 'Strategy', 'Utilities', 'Video Production', 'Web Publishing']
